```
자연어 처리 모델을 만들고
해당 모델을 도커 이미지를생성해서
원하는 곳에 컨테이너로 모델을 실행
```
```
기본과정
  텍스트 전처리
  언어모델
  카운트 기반 단어표현
  벡터의 유사도
  머신러닝
  딥러닝
  순환신경망
  워드  임베딩
  텍스트 분류
  합성곱 신경망
  태깅작업
심화과정
  서브워드 토크나이저
  인코더 디코더
  어텐션 매커니즘
  트랜스포머 모델
  BERT GPT
  한국어 처리  
```

# 1. 텍스트 전처리과정


데이터 준비 : 텍스트 데이터 로드

In [ ]:
text = "안녕하세요! 자연어 처리는 재미있습니다. 😊 하지만 어렵기도 하지요. 데이터 전처리는 중요해요!"

토큰화

In [ ]:
# 단어토큰화(띄어쓰기 기준)
!pip install nltk

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
tokens = word_tokenize(text)
print(tokens)

['안녕하세요', '!', '자연어', '처리는', '재미있습니다', '.', '😊', '하지만', '어렵기도', '하지요', '.', '데이터', '전처리는', '중요해요', '!']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# 형태소 분석기 활용 koNLPy
!pip install koNLPy -q
from konlpy.tag import Okt
okt = Okt()
token_okt = okt.morphs(text)
print(token_okt)

['안녕하세요', '!', '자연어', '처리', '는', '재미있습니다', '.', '😊', '하지만', '어렵', '기도', '하지요', '.', '데이터', '전', '처리', '는', '중요해요', '!']


정제 및 정규화

In [ ]:
import re
text_cleaned = re.sub(r"[^ㄱ-ㅎ가-힣\s]", "", text)  # 한글 및 공백만 남기기
print(text_cleaned)

안녕하세요 자연어 처리는 재미있습니다  하지만 어렵기도 하지요 데이터 전처리는 중요해요


불용어 제거

In [ ]:
stopwords = ['안녕하세요','하지만','전처리는']
tokens_filtered = [word for word in token_okt if word not in stopwords]
print(tokens_filtered)

['!', '자연어', '처리', '는', '재미있습니다', '.', '😊', '어렵', '기도', '하지요', '.', '데이터', '전', '처리', '는', '중요해요', '!']


정규표현식(숫자나 특수문자 제거)

In [ ]:
text_cleaned = re.sub(f'\d+','',text_cleaned)  # 숫자 제거
text_cleaned = re.sub(r'[^\w\s]','',text_cleaned) # 특수문자 제거
print(text_cleaned)

안녕하세요 자연어 처리는 재미있습니다  하지만 어렵기도 하지요 데이터 전처리는 중요해요


# 2. 카운트 기반 단어표현

Bag of Words(BoW)

In [ ]:
# 단어의 등장 횟수를 기반으로 벡터를 만드는 기법
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
    "자연어 처리는 재미있습니다",
    "데이터 전처리는 중요해요",
    "하지만 어렵기도 하지요"
]
X = vectorizer.fit_transform(corpus)
print("단어 사전:", vectorizer.get_feature_names_out())  # 단어 리스트 출력
print("BoW 벡터:\n", X.toarray())  # 단어 등장 횟수 벡터 출력

단어 사전: ['데이터' '어렵기도' '자연어' '재미있습니다' '전처리는' '중요해요' '처리는' '하지만' '하지요']
BoW 벡터:
 [[0 0 1 1 0 0 1 0 0]
 [1 0 0 0 1 1 0 0 0]
 [0 1 0 0 0 0 0 1 1]]


문서-단어 행렬(DTM Document-Term Matrix)

In [ ]:
import pandas as pd

df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,데이터,어렵기도,자연어,재미있습니다,전처리는,중요해요,처리는,하지만,하지요
0,0,0,1,1,0,0,1,0,0
1,1,0,0,0,1,1,0,0,0
2,0,1,0,0,0,0,0,1,1


TF-IDF (Term Frequency - Inverse Document Frequency)
```
TF-IDF는 자주 등장하는 단어에 가중치를 부여하고, 중요하지 않은 단어의 가중치는 줄이는 기법입니다.
```

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(corpus)

df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df_tfidf

,데이터,어렵기도,자연어,재미있습니다,전처리는,중요해요,처리는,하지만,하지요
0,0.00000,0.00000,0.57735,0.57735,0.00000,0.00000,0.57735,0.00000,0.00000
1,0.57735,0.00000,0.00000,0.00000,0.57735,0.57735,0.00000,0.00000,0.00000
2,0.00000,0.57735,0.00000,0.00000,0.00000,0.00000,0.00000,0.57735,0.57735


# 3. 벡터의 유사도 측정

코사인 유사도 (Cosine Similarity)
```
코사인 유사도는 두 벡터 간의 각도를 이용하여 유사도를 측정하는 방법입니다.
0과 1 사이의 값을 가지며, 1에 가까울수록 두 문서가 유사하다는 뜻입니다.
```

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF 행렬을 사용하여 문서 간 코사인 유사도 계산
cos_sim = cosine_similarity(X_tfidf, X_tfidf)

import pandas as pd
df_cos_sim = pd.DataFrame(cos_sim, columns=["문서1", "문서2", "문서3"], index=["문서1", "문서2", "문서3"])
print(df_cos_sim)


     문서1  문서2  문서3
문서1  1.0  0.0  0.0
문서2  0.0  1.0  0.0
문서3  0.0  0.0  1.0


유클리디안 거리 (Euclidean Distance)
```
유클리디안 거리는 두 벡터 사이의 직선 거리(유클리드 거리)를 측정하는 방법입니다.
거리가 작을수록 문서들이 유사하다고 볼 수 있습니다.
```

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

# TF-IDF 행렬을 사용하여 유클리디안 거리 계산
euclidean_dist = euclidean_distances(X_tfidf, X_tfidf)

df_euclidean = pd.DataFrame(euclidean_dist, columns=["문서1", "문서2", "문서3"], index=["문서1", "문서2", "문서3"])
print(df_euclidean)


          문서1       문서2       문서3
문서1  0.000000  1.414214  1.414214
문서2  1.414214  0.000000  1.414214
문서3  1.414214  1.414214  0.000000


# 4. 머신러닝 기반 문서 분석

로지스틱 회귀 (Logistic Regression) → 문서 분류
```
간단한 감성 분석 데이터셋을 가정하겠습니다.
```

In [ ]:
corpus = [
    "이 영화는 정말 재미있어요",  # 긍정
    "너무 지루하고 재미없어요",  # 부정
    "정말 감동적인 영화였어요",  # 긍정
    "별로였어요. 다시 보고 싶지 않아요",  # 부정
    "스토리가 좋고 감동적이었어요",  # 긍정
    "연기가 어색하고 재미없었어요"  # 부정
]

labels = [1, 0, 1, 0, 1, 0]  # 1: 긍정, 0: 부정


1. TF-IDF 벡터화

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


2. 로지스틱 회귀 모델 학습

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X, labels)


LogisticRegression()

 3. 새로운 문장 분류

In [ ]:
new_sentences = ["정말 최고의 영화였습니다", "완전 별로였어요"]
X_test = vectorizer.transform(new_sentences)

predictions = model.predict(X_test)
print(predictions)  # [1, 0] → 첫 번째 문장은 긍정, 두 번째 문장은 부정


[1 0]



K-Means → 문서 군집화

1. K-Means 클러스터링 적용

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 2  # 2개의 그룹으로 클러스터링
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

print("각 문서의 클러스터 할당:", kmeans.labels_)


각 문서의 클러스터 할당: [0 0 0 0 0 1]


 2. 새로운 문서에 대한 클러스터 예측

In [ ]:
new_cluster = kmeans.predict(X_test)
print("새로운 문장의 클러스터:", new_cluster)

새로운 문장의 클러스터: [0 0]


# 5. 딥러닝 기반 문서 분류

LSTM을 활용한 감성 분석

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

corpus = [
    "이 영화는 정말 재미있어요",  # 긍정
    "너무 지루하고 재미없어요",  # 부정
    "정말 감동적인 영화였어요",  # 긍정
    "별로였어요. 다시 보고 싶지 않아요",  # 부정
    "스토리가 좋고 감동적이었어요",  # 긍정
    "연기가 어색하고 재미없었어요"  # 부정
]

labels = np.array([1, 0, 1, 0, 1, 0])  # 1: 긍정, 0: 부정

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
vocab_size = len(tokenizer.word_index) + 1

max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length, padding="post")


2. LSTM 모델 생성 및 학습

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(vocab_size, 16, input_length=max_length),
    LSTM(32),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X, labels, epochs=10, verbose=1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.1667 - loss: 0.6933
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5000 - loss: 0.6927
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8333 - loss: 0.6921
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8333 - loss: 0.6915
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8333 - loss: 0.6908
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 0.6901
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 0.6893
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.6884
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 1.0000 - loss: 0.6875
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 1.0000 - loss: 0.6864


3. 새로운 문장 분류

In [ ]:
new_sentences = ["정말 최고의 영화였습니다", "완전 별로였어요"]
new_sequences = tokenizer.texts_to_sequences(new_sentences)
X_new = pad_sequences(new_sequences, maxlen=max_length, padding="post")

predictions = model.predict(X_new)
print(["긍정" if p > 0.5 else "부정" for p in predictions])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
['긍정', '긍정']


# BERT 기반 문서 분류

1. BERT 모델 로드

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2. 데이터 전처리

In [ ]:
train_encodings = bert_tokenizer(corpus, truncation=True, padding=True, max_length=32, return_tensors="tf")
labels = tf.convert_to_tensor(labels)


In [ ]:
train_encodings

{'input_ids': <tf.Tensor: shape=(6, 15), dtype=int32, numpy=
array([[   101,   9638,  42428,  11018,   9670,  89523,   9659,  22458,
        119192,  12965,  48549,    102,      0,      0,      0],
       [   101,   9004,  32537,   9706,  35866,  12453,   9659,  22458,
        119136,  12965,  48549,    102,      0,      0,      0],
       [   101,   9670,  89523,   8848,  18778,  15387,  42428, 119147,
         12965,  48549,    102,      0,      0,      0,      0],
       [   101,   9353,  11261, 119147,  12965,  48549,    119,  25805,
         98199,   9495,  12508,   9523,  16985,  48549,    102],
       [   101,   9477,  26444,  44130,   9685,  11664,   8848,  18778,
         14801,  10739, 119138,  12965,  48549,    102,      0],
       [   101,   9568,  47869,   9546,  41442,  12453,   9659,  22458,
        119136, 119138,  12965,  48549,    102,      0,      0]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(6, 15), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0

 3. BERT 모델 학습

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# 모델 및 토크나이저 로드
model_name = "bert-base-multilingual-cased"
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터 전처리
corpus = ["이 영화는 정말 재미있어요", "너무 지루하고 재미없어요", "정말 감동적인 영화였어요"]
labels = tf.convert_to_tensor([1, 0, 1])  # 정수형 레이블
train_encodings = bert_tokenizer(corpus, truncation=True, padding=True, max_length=32, return_tensors="tf")

# 손실 함수 및 옵티마이저 (LEGACY 사용)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# 모델 컴파일
bert_model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

# 모델 학습
bert_model.fit(train_encodings["input_ids"], labels, epochs=3, batch_size=2, verbose=1)

# 학습된 모델 저장 및 다시 불러오기 (가중치 초기화 문제 해결)
bert_model.save_pretrained("./bert_model")
bert_model = TFBertForSequenceClassification.from_pretrained("./bert_model")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x7c3c453efe90>

 4. 새로운 문장 분류

In [ ]:
new_sentences = ["정말 최고의 영화였습니다", "완전 별로였어요"]
new_encodings = bert_tokenizer(new_sentences, truncation=True, padding=True, max_length=32, return_tensors="tf")

predictions = bert_model.predict(new_encodings["input_ids"]).logits
print(tf.argmax(predictions, axis=1).numpy())
